
# Inference Notebook (mission15)

이 노트북은 **연구자 2**가 `trainer` 컨테이너가 생성한 산출물로 추론을 수행하고 `result.csv`를 저장하기 위한 것입니다.

- 공유 볼륨 경로: `/shared`
- 필요한 파일: `model.pkl`, `mission15_test.csv`, `metrics.json`

> **실행 순서**
> 1. 경로 설정 및 파일 확인  
> 2. metrics.json 확인  
> 3. 모델 로드  
> 4. 테스트 데이터 로드  
> 5. 예측 및 저장 (`/shared/result.csv`)  


In [9]:

from pathlib import Path

# Docker Compose에서 마운트된 공유 볼륨 경로
SHARED = Path("/shared")

MODEL_PATH = SHARED / "model.pkl"
TEST_PATH  = SHARED / "mission15_test.csv"
METRICS_PATH = SHARED / "metrics.json"  # optional
RESULT_PATH = SHARED / "result.csv"

print("SHARED exists:", SHARED.exists())
print("MODEL_PATH :", MODEL_PATH)
print("TEST_PATH  :", TEST_PATH)
print("RESULT_PATH:", RESULT_PATH)


SHARED exists: True
MODEL_PATH : /shared/model.pkl
TEST_PATH  : /shared/mission15_test.csv
RESULT_PATH: /shared/result.csv


In [10]:

assert MODEL_PATH.exists(), f"모델 파일이 없습니다: {MODEL_PATH}"
assert TEST_PATH.exists(), f"테스트 CSV가 없습니다: {TEST_PATH}"
print("필요한 파일이 모두 존재합니다.")


필요한 파일이 모두 존재합니다.


In [11]:

import json

if METRICS_PATH.exists():
    with open(METRICS_PATH, "r", encoding="utf-8") as f:
        metrics = json.load(f)
    print("metrics.json:")
    for k, v in metrics.items():
        print(f" - {k}: {v}")
else:
    print("metrics.json 파일이 없습니다 (선택 항목).")


metrics.json:
 - holdout_rmse: 2.037492160567813
 - cv_best_rmse: 2.0484412264355543
 - best_alpha: 0.1
 - features: ['Hours Studied', 'Previous Scores', 'Extracurricular Activities', 'Sleep Hours', 'Sample Question Papers Practiced']
 - target: Performance Index
 - refit_full: True


In [12]:

import joblib

model = joblib.load(MODEL_PATH)
print("모델 로드 완료:", type(model))


모델 로드 완료: <class 'sklearn.pipeline.Pipeline'>


In [13]:

import pandas as pd

df_test = pd.read_csv(TEST_PATH)
df_test = df_test.drop_duplicates()
print("test.csv shape:", df_test.shape)
df_test.head()


test.csv shape: (2925, 5)


,Hours Studied,Previous Scores,Extracurricular Activities,Sleep Hours,Sample Question Papers Practiced
0,7,99,Yes,9,1
1,8,51,Yes,7,2
2,8,91,No,4,5
3,5,79,No,7,8
4,2,72,No,4,3


In [14]:

# 학습 시 파이프라인(Pipeline)을 저장했으므로, 동일한 컬럼명을 가진 DataFrame을 그대로 넣을 수 있습니다.
# (train.py의 FEATURES 기준)
FEATURES = [
    'Hours Studied',
    'Previous Scores',
    'Extracurricular Activities',
    'Sleep Hours',
    'Sample Question Papers Practiced'
]

missing = [c for c in FEATURES if c not in df_test.columns]
if missing:
    raise ValueError(f"테스트 데이터에 필요한 컬럼이 없습니다: {missing}")

pred = model.predict(df_test[FEATURES])
print("예측 완료. 샘플 5개:", pred[:5])


예측 완료. 샘플 5개: [91.84892666 45.07144721 84.32554884 65.57910402 47.42539835]


In [16]:

import pandas as pd

out_df = pd.DataFrame({"prediction": pred})
out_df.to_csv(RESULT_PATH, index=False)
print("저장 완료 →", RESULT_PATH)
out_df.head()


저장 완료 → /shared/result.csv


,prediction
0,91.848927
1,45.071447
2,84.325549
3,65.579104
4,47.425398



---
Notebook generated: **2025-08-13T08:58:17**
